# Python3 + Appium + Andriod 东方新闻脚本

脚本功能包括: 签到，金币领取，新闻阅读(仅限广东地区文字新闻)，小视频，视频，其中可以控制这三个功能新闻阅读，小视频，视频的运行时长。

> 东方新闻邀请码，需要的自取：A419886580

### 开发环境配置

[Selenium 安装](https://germey.gitbooks.io/python3webspider/content/1.2.2-Selenium%E7%9A%84%E5%AE%89%E8%A3%85.html)
[appium 安装](https://germey.gitbooks.io/python3webspider/content/1.7.3-Appium%E7%9A%84%E5%AE%89%E8%A3%85.html)


### Appium基础

- platformName: 手机操作系统
- deviceName: 手机类型
将andriod手机链接电脑后，电脑打开Appium，同时手机打开USB调试功能，可通过 `adb devices -l` 命令来测试链接状态，如果出现类似结果，说明PC已经正确连接手机。

```
List of devices attached
177dca8c               device product:c7ltezc model:SM_C7000 device:c7ltechn transport_id:3
```

- newCommandTimeout: 两条appium命令间的最长时间间隔，若超过这个时间，appium会自动结束并退出app

- noReset, fullReset: noReset 不要在会话前重置应用状态。默认值false。 fullReset (Android) 通过卸载而不是清空数据来重置应用状态。在Android上, 这也会在会话结束后自动清除被测应用。默认值false。【建议用 noReset, 可以保存设置中的登录状态，这样滤过复杂的登录验证】

- appActivity, appPackage: appActivity与appPackage指用于启动待测app的activityName与packageName


### 东方新闻

- 东方新闻版本：V30928000

- 东方新闻的元素大概3-4天就会更换一次，所以脚本中涉及 resource_id 元素处需要不定期修改。

- 为了减少封账号的风险，脚本里面采用的时间、滑动位置方法都是随机的。

- 经测试，基本上每天刷上个1小时，后面的金币的量就微乎其微了。

- 由于弹出框弹出不可控，没有完全测全。



### 使用脚本

1. 打开Appium
2. 开启 Appium Server
3. 运行如下脚本

In [ ]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time, random, datetime, re
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

# Appium 基本参数
PLATFORM = 'Android'
DEVICE_NAME = 'SM_C7000'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 600
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700


class Automation():

    # 初始化 Appium 基本参数
    def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.desired_caps = {
            'platformName': PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER, self.desired_caps)
        self.wait = WebDriverWait(self.driver, 10)
        self.size = self.driver.get_window_size()

    # 屏幕方法
    def swipeUp(self):
        # 向上滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.11, 0.25), random.uniform(800, 1200))
        print('向上滑动屏幕')

    def swipeDown(self):
        # 向下滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.11, 0.25),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.75, 0.89), random.uniform(800, 1200))
        print('向下滑动屏幕')

    def swipeRight(self):
        # 向右滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.01, 0.11),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.89, 0.98),
                          self.size['height'] * random.uniform(0.75, 0.89), random.uniform(800, 1200))
        print('向右滑动屏幕')


class Eastnews(Automation):

    def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.popup_ele = 'com.songheng.eastnews:id/a_t'
        self.popupcoin_ele = 'com.songheng.eastnews:id/go'
        self.popupnews_ele = 'com.songheng.eastnews:id/wf'
        self.home_ele = 'com.songheng.eastnews:id/jo'
        self.coin_ele = 'com.songheng.eastnews:id/aqj'
        self.mission_ele = 'com.songheng.eastnews:id/ju'
        self.mpopup_ele = 'com.songheng.eastnews:id/u9'
        self.siginstatus_ele = '//*[@text="已领取"]'
        self.ignore_ele = '//*[@text="忽略"]'
        self.signin_ele = 'signIn'
        self.video_ele = 'com.songheng.eastnews:id/js'
        self.videohome_ele = '//*[@text="推荐"]'
        self.videoclose_ele = 'com.songheng.eastnews:id/k0'
        self.content_ele = 'com.songheng.eastnews:id/a29'
        self.play_ele = 'com.songheng.eastnews:id/pb'
        self.time_text_ele = 'com.jifen.qukan:id/a45'
        self.svideo_ele = '//*[@text="小视频"]'
        self.svideoitem_ele = 'com.songheng.eastnews:id/a9_'
        self.gdmenu_ele = '//*[@text="广东"]'
        self.reward_ele = 'com.songheng.eastnews:id/aod'
        self.adclose_ele = 'com.songheng.eastnews:id/f7'
        self.articles_ele = 'com.songheng.eastnews:id/op'
        self.return_ele = 'com.songheng.eastnews:id/ey'
        self.my_ele = 'com.songheng.eastnews:id/jv'
        self.mypopup_ele = 'com.songheng.eastnews:id/wt'
        self.lottery_ele = '//*[@text="幸运转盘"]'
        self.chance_ele = '/hierarchy/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.support.v4.widget.SlidingPaneLayout/android.widget.FrameLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.webkit.WebView/android.webkit.WebView/android.view.View/android.view.View[5]'
        self.lotterybtn_ele = '/hierarchy/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.support.v4.widget.SlidingPaneLayout/android.widget.FrameLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.webkit.WebView/android.webkit.WebView/android.view.View/android.view.View[6]/android.view.View[1]/android.widget.Image'
        self.nogiftclose_ele = 'com.songheng.eastnews:id/aos'
        self.gainbtn_ele = 'gainBtn'
        self.gainclose_ele = 'com.songheng.eastnews:id/tt_video_ad_close'
        self.windowclose_ele = 'com.songheng.eastnews:id/go'
        self.returnbtn_ele = 'com.songheng.eastnews:id/k0'
        super().__init__(APP_PACKAGE, APP_ACTIVITY)

    def landing_page(self):
        time.sleep(random.randint(5,7))
        try:
            popup_coin = self.wait.until(EC.presence_of_element_located((By.ID, self.popupcoin_ele)))
            print('金币领取通知')
            popup_coin.click()
            print('关闭金币领取通知')
        except:
            print('无金币领取')
        try:
            popup_news = self.wait.until(EC.presence_of_element_located((By.ID, self.popupnews_ele)))
            print('新闻推送弹出框')
            popup_news.click()
            print('关闭新闻推送弹出框')
        except:
            print('无新闻推送弹出框')
        self.driver.tap([(851, 156), (852, 157)], 300)
        print('tap anywhere')

    # 领金币
    def get_coins(self):
        home = self.wait.until(EC.presence_of_element_located((By.ID, self.home_ele)))
        home.click()
        print('点击底部菜单:新闻...')
        coins = self.wait.until(EC.presence_of_element_located((By.ID, self.coin_ele)))
        time.sleep(2)
        if coins.get_attribute('text') == '领取':
            coins.click()
            print('领取金币')
            time.sleep(1)
            windowclose_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.windowclose_ele)))
            windowclose_btn.click()
            print('关闭弹出框')
        else:
            print('无金币可领取')

    # 每日签到
    def mission(self):
        mission = self.wait.until(EC.presence_of_element_located((By.ID, self.mission_ele)))
        mission.click()
        print('点击底部菜单:任务...')
        try:
            mission_popup = self.wait.until(EC.presence_of_element_located((By.ID, self.mpopup_ele)))
            mission_popup.click()
            print('取消宝箱弹窗')
        except:
            print('无宝箱弹窗')
        try:
            time.sleep(2)
            signin_status = self.wait.until(EC.presence_of_element_located((By.XPATH, self.siginstatus_ele)))
            print('已签到')
        except:
            print('未签到')
            signin_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.signin_ele)))
            signin_btn.click()
            print('已签到...')
            time.sleep(2)

    # 视频
    def video(self, interval):
        video = self.wait.until(EC.presence_of_element_located((By.ID, self.video_ele)))
        video.click()
        print('点击底部菜单:视频...')
        video_home = self.wait.until(EC.presence_of_element_located((By.XPATH, self.videohome_ele)))
        video_home.click()
        print('点击顶部菜单:推荐...')
        t = 1
        start = time.time()
        while True:
            content = self.driver.find_elements_by_id(self.content_ele)
            print('视频数量', len(content))
            try:
                for i in range(len(content)):
                    video_paly = self.driver.find_element_by_id(self.play_ele)
                    print('第{}次进入循环.. 点击第{}视频'.format(t, i + 1))
                    video_paly.click()
                    time.sleep(random.randint(100, 120))
                    #self.swipeRight()
                    return_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.returnbtn_ele)))
                    return_btn.click()
                    print('退出视频')
            except :
                print('广告')
            t += 1
            self.swipeUp()
            time.sleep(random.randint(5, 20))
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束视频循环')
                break

    # 小视频
    def smallvideo(self, interval):
        video = self.wait.until(EC.presence_of_element_located((By.ID, self.video_ele)))
        video.click()
        print('点击底部菜单:视频...')
        smallvideo = self.wait.until(EC.presence_of_element_located((By.XPATH, self.svideo_ele)))
        smallvideo.click()
        print('点击顶部菜单:小视频...')
        smallvideo_item = self.wait.until(EC.presence_of_element_located((By.ID, self.svideoitem_ele)))
        smallvideo_item.click()
        print('进入小视频...')
        start = time.time()
        while True:
            time.sleep(random.randint(5, 20))
            self.swipeUp()
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束小视频循环')
                break
        windowclose_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.windowclose_ele)))
        windowclose_btn.click()
        print('退出小视频')

    # 阅读新闻
    def read_article(self, interval):
        home = self.wait.until(EC.presence_of_element_located((By.ID, self.home_ele)))
        home.click()
        print('点击底部菜单:新闻...')
        sz_menu = self.wait.until(EC.presence_of_element_located((By.XPATH, self.gdmenu_ele)))
        sz_menu.click()
        print('点击顶部菜单:广东...')
        start = time.time()
        while True:
            articles = self.wait.until(EC.presence_of_all_elements_located((By.ID, self.articles_ele)))
            print('新闻数量', len(articles))
            for article in articles:
                print('进入新闻：{}'.format(article.get_attribute('text')))
                article.click()
                try:
                    reward_icon = self.wait.until(EC.presence_of_element_located((By.ID, self.reward_ele)))
                    for i in range(random.randint(2, 6)):
                        time.sleep(random.randint(3, 7))
                        self.swipeUp()
                    for i in range(random.randint(2, 6)):
                        time.sleep(random.randint(3, 7))
                        self.swipeDown()
                    return_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.return_ele)))
                except:
                    print('此为广告，直接退出')
                    return_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.adclose_ele)))
                return_btn.click()
                print('退出进入菜单')

            for i in range(random.randint(2, 4)):
                time.sleep(random.randint(1, 3))
                self.swipeUp()
                print('菜单深圳')
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束新闻循环')
                break
            else:
                print('未到结束时间')

    def lottery(self):
        my_menu = self.wait.until(EC.presence_of_element_located((By.ID, self.my_ele)))
        my_menu.click()
        print('点击底部菜单:我的...')
        try:
            my_popup = self.wait.until(EC.presence_of_element_located((By.ID, self.mypopup_ele)))
            my_popup.click()
            print('取消弹窗')
        except:
            print('无弹窗')
        lottery = self.wait.until(EC.presence_of_element_located((By.XPATH, self.lottery_ele)))
        lottery.click()
        print('点击抽奖菜单')
        time.sleep(random.randint(4, 6))
        chances = self.wait.until(EC.presence_of_element_located((By.XPATH, self.chance_ele))).get_attribute('text')
        chance_num = re.search(r'剩余次数 (\d+)', chances).group(1)
        print('剩余抽奖次数 {}'.format(chance_num))
        for i in range(int(chance_num)):
            time.sleep(random.randint(1, 3))
            lottery_btn = self.wait.until(EC.presence_of_element_located((By.XPATH, self.lotterybtn_ele)))
            lottery_btn.click()
            print('第{}次抽奖'.format(i + 1))
            time.sleep(random.randint(1, 3))
            try:
                nogift_close = self.wait.until(EC.presence_of_element_located((By.ID, self.nogiftclose_ele)))
                nogift_close.click()
                print('没有奖品')
            except:
                gain_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.gainbtn_ele)))
                gain_btn.click()
                print('领取奖品')
                try:
                    time.sleep(10)
                    try:
                        gainclose_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.gainclose_ele)))
                    except:
                        gainclose_btn = self.wait.until(EC.presence_of_element_located((By.ID, self.windowclose_ele)))
                    print('奖品为其他')
                    gainclose_btn.click()
                    lottery_btn = self.wait.until(EC.presence_of_element_located((By.XPATH, self.lotterybtn_ele)))
                    lottery_btn.click()
                except:
                    print('奖品为金币')
        print('抽奖结束')

    # 入口函数
    def main(self):
        print('打开 {} App ...'.format('东方头条'))
        try:
            self.landing_page()
            self.mission()
            func_list = ['smallvideo', 'read_article', 'video']  # 新闻阅读，小视频，视频三个方法随机
            for i in range(12):
                self.get_coins()
                exp = 'self.{}({})'.format(random.choice(func_list), random.randint(500, 700))
                print(exp)
                eval(exp)
            # self.lottery()
        except TimeoutException:
            ignore_btn = self.wait.until(EC.presence_of_element_located((By.XPATH, self.ignore_ele)))
            ignore_btn.click()
            print('要闻推送取消')

        print('App {} 今日任务完成！'.format('东方头条'))

eastnews_appPackage = 'com.songheng.eastnews'
eastnews_appActivity = 'com.oa.eastfirst.activity.WelcomeActivity'

if __name__ == '__main__':
    eastnews = Eastnews(eastnews_appPackage, eastnews_appActivity)
    eastnews.main()

- 新闻要闻弹窗没解决